<a href="https://colab.research.google.com/github/markbozsoki/ColabNotebooks/blob/main/NASA_Mars_Landmark_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>NASA Mars Landmarks Classification

https://www.kaggle.com/datasets/sshikamaru/mars-landmarks

In [1]:
# IMPORTS

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from collections import Counter

In [2]:
# DATA ACCESS

print("Initializing Kaggle...")
!pip install -q kaggle
!mkdir -p ~/.kaggle

print("\nInitializing Kaggle API access...")
!pip install --upgrade --no-cache-dir -q gdown
!gdown 1m1sgCgeHHtjeeTtSKjZmFg-CRnuEIEHL -q --output ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

print("\nDownloading dataset...")
!kaggle datasets download --force -q -d sshikamaru/mars-landmarks


print("\nUnzipping downloaded dataset...")
!unzip -q -o mars-landmarks.zip -d mars-landmarks

print("\nDone!")

Initializing Kaggle...

Initializing Kaggle API access...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


Unzipping downloaded dataset...

Done!


In [3]:
classmap_csv = pd.read_csv("mars-landmarks/landmarks_map-proj-v3_classmap.csv", header=None, names=["Class_ID", "Class_Name"])
classmap_csv

,Class_ID,Class_Name
0,0,other
1,1,crater
2,2,dark dune
3,3,slope streak
4,4,bright dune
5,5,impact ejecta
6,6,swiss cheese
7,7,spider


In [5]:
labels_csv = pd.read_csv("mars-landmarks/labels-map-proj-v3.txt", sep=' ', header=None, names=["Image", "Class_ID"])
labels_csv = labels_csv.merge(classmap_csv, how="inner", on="Class_ID")
labels_csv

,Image,Class_ID,Class_Name
0,ESP_011623_2100_RED-0069.jpg,0,other
1,ESP_011623_2100_RED-0069-r90.jpg,0,other
2,ESP_011623_2100_RED-0069-r180.jpg,0,other
3,ESP_011623_2100_RED-0069-r270.jpg,0,other
4,ESP_011623_2100_RED-0069-fh.jpg,0,other
...,...,...,...
73026,ESP_016758_2570_RED-0037-r180.jpg,4,bright dune
73027,ESP_016758_2570_RED-0037-r270.jpg,4,bright dune
73028,ESP_016758_2570_RED-0037-fh.jpg,4,bright dune
73029,ESP_016758_2570_RED-0037-fv.jpg,4,bright dune
